In [2]:
! pip install scikit-learn
import sklearn

In [55]:
# import the library package 
import pandas 
import numpy 

from sklearn.feature_selection import RFE 
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor 

import matplotlib.pyplot as plt 

from sklearn.metrics import mean_absolute_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import cross_val_score

# create your MLP in keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import KFold
from keras.optimizers import Adam

# fix random seed for reproducibility 
seed = 7
numpy.random.seed(seed)


In [56]:
# import the dataset
forestfires = pd.read_csv("forestfires.csv")

# encode the dataset
forestfires.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
forestfires.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)

In [57]:
# print the dataset
print("Head:",forestfires.head)

Head: <bound method NDFrame.head of      month  day  FFMC    DMC  ...  monthnov  monthoct  monthsep  size_category
0        3    5  86.2   26.2  ...         0         0         0          small
1       10    2  90.6   35.4  ...         0         1         0          small
2       10    6  90.6   43.7  ...         0         1         0          small
3        3    5  91.7   33.3  ...         0         0         0          small
4        3    7  89.3   51.3  ...         0         0         0          small
..     ...  ...   ...    ...  ...       ...       ...       ...            ...
512      8    7  81.6   56.7  ...         0         0         0          large
513      8    7  81.6   56.7  ...         0         0         0          large
514      8    7  81.6   56.7  ...         0         0         0          large
515      8    6  94.4  146.0  ...         0         0         0          small
516     11    2  79.5    3.0  ...         1         0         0          small

[517 rows x 31 

In [58]:
forestfires.describe()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,7.475822,4.259188,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,0.143133,0.162476,0.183752,0.117988,0.123791,0.104449,0.017408,0.355899,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,2.275990,2.072929,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,0.350548,0.369244,0.387657,0.322907,0.329662,0.306138,0.130913,0.479249,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,1.000000,1.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,2.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.000000,5.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,6.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,12.000000,7.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [59]:
forestfires.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    int64  
 1   day            517 non-null    int64  
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [60]:
forestfires.shape

(517, 31)

In [61]:
forestfires.dtypes # datatypes

month              int64
day                int64
FFMC             float64
DMC              float64
DC               float64
ISI              float64
temp             float64
RH                 int64
wind             float64
rain             float64
area             float64
dayfri             int64
daymon             int64
daysat             int64
daysun             int64
daythu             int64
daytue             int64
daywed             int64
monthapr           int64
monthaug           int64
monthdec           int64
monthfeb           int64
monthjan           int64
monthjul           int64
monthjun           int64
monthmar           int64
monthmay           int64
monthnov           int64
monthoct           int64
monthsep           int64
size_category     object
dtype: object

In [62]:
print("correlation:",forestfires.corr(method='pearson'))

correlation:              month       day      FFMC  ...  monthnov  monthoct  monthsep
month     1.000000 -0.050837  0.291477  ...  0.068231  0.191895  0.473304
day      -0.050837  1.000000 -0.041068  ... -0.048025 -0.032764 -0.022955
FFMC      0.291477 -0.041068  1.000000  ... -0.088964 -0.005998  0.076609
DMC       0.466645  0.062870  0.382619  ... -0.074218 -0.187632  0.110907
DC        0.868698  0.000105  0.330512  ... -0.078380  0.093279  0.531857
ISI       0.186597  0.032909  0.531805  ... -0.076559 -0.071154 -0.068877
temp      0.368842  0.052190  0.431532  ... -0.053798 -0.053513  0.088006
RH       -0.095280  0.092151 -0.300995  ... -0.035885 -0.072334 -0.062596
wind     -0.086368  0.032478 -0.028485  ...  0.011864 -0.053850 -0.181476
rain      0.013438 -0.048340  0.056702  ... -0.003225 -0.012665 -0.051733
area      0.056496  0.023226  0.040122  ... -0.008893 -0.016878  0.056573
dayfri   -0.028562  0.158676  0.019306  ... -0.019527 -0.045585  0.107671
daymon   -0.002941 -0.643

In [63]:
forestfires = forestfires.values

# split into input and output values 
x=forestfires[:,0:12]
y=forestfires[:,12]

In [64]:
# feature selection 
model = ExtraTreesRegressor()
rfe = RFE(model,3)
model1 = rfe.fit(x,y)

print("Numbers of features:",model1.n_features_)
print("Selected features:",model1.support_)
print("Feature ranking :",model1.ranking_)

Numbers of features: 3
Selected features: [False  True False False False False False False False False  True  True]
Feature ranking : [10  1  8  9  7  6  5  4  3  2  1  1]


In [65]:
# normalize the dataset
Y = numpy.array(y).reshape((len(y), 1))


scaler = MinMaxScaler(feature_range=(0, 1))
Y = scaler.fit_transform(Y)

In [75]:
# define model 
def create_model():
  model = Sequential()
  model.add(Dense(10,input_dim = 12,kernel_initializer='uniform',activation='relu'))
  model.add(Dense(3,kernel_initializer='uniform',activation='relu'))
  model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))

  adam=Adam(learning_rate=0.01)

  # compile model 
  model.compile (loss='mean_squared_error',optimizer = adam)
  return model

In [76]:
# evaluate the model 
estimator = KerasRegressor(build_fn=create_model,epochs=600,batch_size=5,verbose = 0)
kfold = KFold(n_splits=30,random_state=seed)
results = cross_val_score(estimator,x,y,cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

  FitFailedWarning)


Results: nan (nan) MSE
